In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import seaborn
from pandas.plotting import parallel_coordinates
import datetime as dt
from scipy import stats
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

%matplotlib inline

In [ ]:
pd.set_option('display.max_columns', None)


In [ ]:
data = pd.read_csv('weather_crime.csv')


In [ ]:
data.head()

In [ ]:
data.rename(columns = {'date__of_occurrence':'date_of_occurrence','_primary_description':'primary_description','_secondary_description':'secondary_description','_location_description':'location_description','month_x':'month','day_x':'day','temperaturehigh':'high_temp','temperaturelow':'low_temp'}, inplace = True)


In [ ]:
data.head()

In [ ]:
data = data.drop(['sunrisetime','sunsettime','temperaturemax','temperaturemin'],axis=1)

In [ ]:
data = data.drop(['Unnamed: 0'],axis=1)

In [ ]:
data

In [ ]:
crime_count = pd.DataFrame(data.groupby('primary_description').size().sort_values(ascending=False).rename('counts').reset_index())

crime_count.head(31)

In [ ]:
data.columns

In [ ]:
import datetime

In [ ]:
datem = datetime.datetime.strptime(, "%Y-%m-%d %H:%M:%S")
print(datem.year)       # 2021
print(datem.hour)       # 11

In [ ]:
#Check multicolinearity:
correlations_matrix = data.corr()
correlations_matrix

sns.heatmap(correlations_matrix, annot=True)
plt.show()

In [ ]:
#Attributes of primary type is plotted in the form of bar to find and analyse highest type crime activity done form 2012 to 2017

import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="whitegrid")

f, ax = plt.subplots(figsize=(10, 15))
sns.set_color_codes("pastel")
sns.barplot(y="primary_description", x="counts", data=crime_count.iloc[:10, :],label="Total", color="grey")

ax.legend(ncol=2, loc="lower right", frameon=True)
ax.set(ylabel="Type",xlabel="Crimes")
sns.despine(left=True, bottom=True)
plt.show()

In [ ]:
b = pd.DataFrame(data ,columns=['primary_description','high_temp'])
b

In [ ]:
violent = ['ASSAULT','BATTERY','CRIM SEXUAL ASSAULT', 'ROBBERY',
           'SEX OFFENSE', 'HOMICIDE', 'INTIMIDATION', 'KIDNAPPING', 'SEX']

In [ ]:
# weather = data.loc[date_mask]
# crime_chi = crime_chi.loc[date_mask2]

In [ ]:
# total_crimes = (crime_chi.groupby(['date']).size())
# avg_weather = weather.groupby(['date']).mean()
# # print(avg_weather)
# # print(total_crimes)

In [ ]:
avg_weather = data.groupby(['date']).mean()

In [ ]:
avg_weather

In [ ]:
# drop rows with 'other offense' as primary description



In [ ]:
data.isnull().values.any()
regr = LinearRegression()
# std_dev = 4
#getting rid of data that is outside 4 standard deviations.
# result = result[(np.abs(stats.zscore(result)) < float(std_dev)).all(axis=1)]
X = pd.DataFrame(data['high_temp'])
y = pd.DataFrame(data['primary_description'])
# print(y)
regr.fit(X,y)
print(regr.coef_)
print(regr.intercept_)
pred = regr.predict(X)
plt.scatter(data['high_temp'], data['primary_description'], color='g')
plt.plot(X, pred, color='b')
plt.show()

X2 = pd.DataFrame(data['precipintensity'])
regr2 = LinearRegression()
regr2.fit(X2, y)
p2 = regr2.predict(X2)
print(regr2.coef_)
print(regr2.intercept_)
plt.scatter(data['precipintensity'], data['violent'], color='k')
plt.plot(X2, p2, color='y')
plt.show()

In [ ]:
# date = pd.date_range(start=start_date, end=end_date)
# new_df = pd.DataFrame({'Date':date, 'Incidents':total_crimes}, )
# new_df

#create a merged data frame?
frames = [data, avg_weather]
result = pd.concat(frames, axis=1, sort=True)
result = result.dropna()
result.columns = ['Incidents','Temp (C)', 'Total Precipitation (mm)']
result.plot(kind='Scatter', x='Temp (C)', y='Incidents')
result.plot(kind='Scatter', x='Total Precipitation (mm)', y='Incidents', color='r')
result.plot()

In [ ]:
# Create bins for temperature data
temp_bins = [-100, -20, -10, 0, 10, 20, 30, 100]
temp_labels = ["-100 to -20", "-20 to -10" "-10 to 0", "0 to 10", "10 to 20", "20 to 30", "30 to 100"]

# Bin the Temp column and add it to the dataframe as the Temp. Bin column
weather_crime_df['Temp Bin'] = pd.cut(weather_crime_df['Temp'], temp_bins, labels=temp_labels)

# Group by temperature bins, counting the number of crimes committed in each bucket
grouped_by_temp_bins = weather_crime_df.groupby(['Temp Bin', 'Type'])['Date'].count()

grouped_by_temp_bins_ = weather_crime_df.groupby(['Temp Bin', 'Type'])['Date'].count()

# Create a Dataframe with the grouped temperature data and reset index for plotting
grouped_by_temp_bins_df = pd.DataFrame(grouped_by_temp_bins.reset_index())

# Rename columns
grouped_by_temp_bins_df.columns = ['Temp Bin', 'Crime Type', 'Number of Crimes']

# Pivot Dataframe as preparation for plotting
grouped_by_temp_bins_df2 = grouped_by_temp_bins_df.pivot(index='Temp Bin', columns='Crime Type', values = 'Number of Crimes')

# Preview Dataframe
grouped_by_temp_bins_df2

In [ ]:
for cat_var in ['month', 'hour', 'weekday', 'Domestic', 'Arrest', 'District', 'Ward', 'Location Description']:
    if cat_var == 'weekday':
        # Sort by week
        order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    elif cat_var in ['District', 'Ward', 'Location Description']:
        # Sort by frequency
        order = data.groupby(by=cat_var).size().sort_values(ascending=False).index
    else:
        # Do not sort explicitly
        order = None
    seaborn.catplot(data=data, x=cat_var, kind='count', order=order)
    plt.xticks(rotation=90)